In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from os.path import join
from os import getcwd
from IPython.display import clear_output

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, auc

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, Adam
from keras.metrics import AUC # Area under the curve, default: ROC
from keras.losses import BinaryCrossentropy
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, TensorBoard
from keras.optimizers.schedules import ExponentialDecay
from keras.initializers import GlorotNormal
import kerastuner as kt

In [4]:
from functions import *

In [5]:
checkpoints_path = getcwd()+'\\checkpoints'

In [6]:
df = pd.read_csv('../../databases/diabetes.csv')
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
outlayers = {
    'BloodPressure': (40, np.Inf),
    'SkinThickness': (0, 80),
    'Insulin': (0, 400),
    'BMI': (0, 50)
}

zeros = [
    'Glucose',
    'BloodPressure',
    'SkinThickness',
    'Insulin',
    'BMI'
]
x_df = df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI', 'DiabetesPedigreeFunction', 'Age']]
y_df = df['Outcome']

In [8]:
# Split dataset into 15% test, 85% train 
x_temp, x_test, y_temp, y_test = train_test_split(x_df, y_df, test_size=0.15, random_state=1)
x_train, x_valid, y_train, y_valid = train_test_split(x_temp, y_temp, test_size=0.15, random_state=1)

# Regresión Logistica

### Con los datos sin procesar

In [9]:
model_0 = Sequential()
model_0.add(Dense(units=1, activation='sigmoid', input_shape=(x_train.shape[1],)))
model_0.compile(optimizer=SGD(), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

In [10]:
model_0_checkpoint_callback = ModelCheckpoint(
    filepath=join(checkpoints_path, 'basic'),
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True)

In [11]:
%%time
history = model_0.fit(x_train, y_train, epochs=50, 
            validation_data=(x_valid, y_valid),
            callbacks=[model_0_checkpoint_callback],
            verbose=0)
# Cargo el mejor modelo entrenado
model_0.load_weights(join(checkpoints_path, 'basic'))
verify_model(model_0, x_train, y_train, x_valid, y_valid)

Wall time: 4.09 s


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.653173,0.423823,0.818653,0.431694,0.813830
1,Validacion,0.728401,0.500000,0.911765,0.492063,0.914286


### Reemplazando datos invalidos

In [12]:
x_train_nonzero, _data = replace_outliers_zeros(x_train, {}, zeros, mean_median=True)
x_test_nonzero, _data = replace_outliers_zeros(x_test, {}, zeros, mean_median=True, data_to_replace=_data)
x_valid_nonzero, _data = replace_outliers_zeros(x_valid, {}, zeros, mean_median=True, data_to_replace=_data)

In [13]:
model_1 = Sequential()
model_1.add(Dense(units=1, activation='sigmoid', input_shape=(x_train.shape[1],)))
model_1.compile(optimizer=SGD(), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

model_1_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'non_zero'),
                                              save_weights_only=True,
                                              monitor='val_auc',
                                              mode='max',
                                              save_best_only=True)

In [14]:
%%time 
history = model_1.fit(x_train_nonzero, y_train, epochs=50, validation_data=(x_valid_nonzero, y_valid), callbacks=[model_1_checkpoint_callback], verbose=0)
# Cargo el mejor modelo entrenado
model_1.load_weights(join(checkpoints_path, 'non_zero'))
verify_model(model_1, x_train_nonzero, y_train, x_valid_nonzero, y_valid)

Wall time: 3.06 s


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.687870,0.778393,0.538860,0.565217,0.759459
1,Validacion,0.736673,0.890625,0.558824,0.730769,0.791667


Se puede observar como en ambos casos se obtuvo un mejor resultado en validacion que en train, lo cual es un comportamiento que se puede considerar poco esperado

### Reemplazando outlayers

In [15]:
x_train_clean, _data = replace_outliers_zeros(x_train, outlayers, zeros, mean_median=True)
x_test_clean, _data = replace_outliers_zeros(x_test, outlayers, zeros, mean_median=True, data_to_replace=_data)
x_valid_clean, _data = replace_outliers_zeros(x_valid, outlayers, zeros, mean_median=True, data_to_replace=_data)

In [16]:
model_2 = Sequential()
model_2.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_clean.shape[1],)))
model_2.compile(optimizer=SGD(), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

model_2_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'clean'),
                                              save_weights_only=True,
                                              monitor='val_auc',
                                              mode='max',
                                              save_best_only=True)

In [17]:
%%time 
history = model_2.fit(x_train_clean, y_train, epochs=50, validation_data=(x_valid_clean, y_valid), callbacks=[model_2_checkpoint_callback], verbose=0)
# Cargo el mejor modelo entrenado
model_2.load_weights(join(checkpoints_path, 'clean'))
verify_model(model_2, x_train_clean, y_train, x_valid_clean, y_valid)

Wall time: 3.06 s


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.696102,0.700831,0.626943,0.528384,0.778462
1,Validacion,0.741268,0.718750,0.617647,0.538462,0.779661


### Normalizando los datos de entrada

In [18]:
x_train_norm, _norm_dict = normalize(x_train_clean, None)
x_valid_norm, _norm_dict = normalize(x_valid_clean, _norm_dict)
x_test_norm, _norm_dict = normalize(x_test_clean, _norm_dict)

In [19]:
model_3 = Sequential()
model_3.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
model_3.compile(optimizer=SGD(), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

model_3_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'norm'),
                                              save_weights_only=True,
                                              monitor='val_auc',
                                              mode='max',
                                              save_best_only=True)

In [20]:
%%time 
history = model_3.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[model_3_checkpoint_callback], verbose=1)
# Cargo el mejor modelo entrenado
model_3.load_weights(join(checkpoints_path, 'norm'))
verify_model(model_3, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/100
18/18 [==============================] - 1s 33ms/step - loss: 0.9172 - auc: 0.5079 - val_loss: 0.7171 - val_auc: 0.6749
Epoch 2/100
18/18 [==============================] - 0s 13ms/step - loss: 0.8702 - auc: 0.5289 - val_loss: 0.6846 - val_auc: 0.6955
Epoch 3/100
18/18 [==============================] - 0s 14ms/step - loss: 0.8285 - auc: 0.5521 - val_loss: 0.6555 - val_auc: 0.7123
Epoch 4/100
18/18 [==============================] - 0s 13ms/step - loss: 0.7907 - auc: 0.5773 - val_loss: 0.6305 - val_auc: 0.7302
Epoch 5/100
18/18 [==============================] - 0s 13ms/step - loss: 0.7574 - auc: 0.6012 - val_loss: 0.6090 - val_auc: 0.7470
Epoch 6/100
18/18 [==============================] - 0s 14ms/step - loss: 0.7284 - auc: 0.6241 - val_loss: 0.5901 - val_auc: 0.7601
Epoch 7/100
18/18 [==============================] - 0s 18ms/step - loss: 0.7028 - auc: 0.6456 - val_loss: 0.5738 - val_auc: 0.7677
Epoch 8/100
18/18 [==============================] - 0s 14ms/step - loss: 0.

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.804673,0.814404,0.616580,0.639785,0.798913
1,Validacion,0.845588,0.859375,0.588235,0.689655,0.797101


### Early Stopping
Los hyperparámetrso principales de este callback son:
- monitor: indica la variable a monitorear
- min_delta: la minima diferencia que se considera como mejora
- patience: cantidad de epochs sin mejoras antes de parar

In [21]:
stop_early = EarlyStopping(monitor='val_auc', patience=20)

In [22]:
%%time 
history = model_3.fit(x_train_norm, y_train, epochs=50, validation_data=(x_valid_norm, y_valid), callbacks=[model_3_checkpoint_callback, stop_early], verbose=1)
# Cargo el mejor modelo entrenado
model_3.load_weights(join(checkpoints_path, 'norm'))
verify_model(model_3, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/50
18/18 [==============================] - 0s 3ms/step - loss: 0.5185 - auc: 0.8054 - val_loss: 0.4703 - val_auc: 0.8451
Epoch 2/50
18/18 [==============================] - 0s 2ms/step - loss: 0.5156 - auc: 0.8072 - val_loss: 0.4692 - val_auc: 0.8447
Epoch 3/50
18/18 [==============================] - 0s 2ms/step - loss: 0.5129 - auc: 0.8093 - val_loss: 0.4683 - val_auc: 0.8438
Epoch 4/50
18/18 [==============================] - 0s 2ms/step - loss: 0.5103 - auc: 0.8115 - val_loss: 0.4674 - val_auc: 0.8433
Epoch 5/50
18/18 [==============================] - 0s 2ms/step - loss: 0.5079 - auc: 0.8128 - val_loss: 0.4667 - val_auc: 0.8449
Epoch 6/50
18/18 [==============================] - 0s 2ms/step - loss: 0.5056 - auc: 0.8143 - val_loss: 0.4662 - val_auc: 0.8454
Epoch 7/50
18/18 [==============================] - 0s 2ms/step - loss: 0.5037 - auc: 0.8161 - val_loss: 0.4657 - val_auc: 0.8447
Epoch 8/50
18/18 [==============================] - 0s 2ms/step - loss: 0.5019 - auc: 0.81

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.804673,0.814404,0.616580,0.639785,0.798913
1,Validacion,0.845588,0.859375,0.588235,0.689655,0.797101


Se puede evidenciar como al utilizar early stopping el entrenamiento finaliza antes debido a que las mejoras son reducidas

## Ajuste de Hyperparámetros

### Learning rate
Por defecto en SGD el learning rate que utiliza keras es 0.01

In [23]:
learning_rates = [1e-5, 1e-4, 1e-3, 0.01, 0.05, 0.1, 0.22, 0.33, 0.47, 0.56, 0.81, 1.2, 1.8, 2.2, 2.7, 3.3, 4.7]
learning_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'learning'),
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)

arr_of_metrics = []
auc_res = {'L_r':['AUC', 'Epochs']}
for i in learning_rates:
    learning = Sequential()
    learning.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
    learning.compile(optimizer=SGD(learning_rate=i), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

    history = learning.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[learning_checkpoint_callback, stop_early], 
                          verbose=1)
    # Cargo el mejor modelo entrenado
    learning.load_weights(join(checkpoints_path, 'learning'))
    metrics = verify_model(learning, x_train_norm, y_train, x_valid_norm, y_valid)
    arr_of_metrics.append(metrics)
    auc_res[i] = [metrics['AUC ROC'][1], history.epoch[-1]]
    clear_output(wait=True)
pd.DataFrame(data=auc_res)

,L_r,1e-05,0.0001,0.001,0.01,0.05,0.1,0.22,0.33,0.47,0.56,0.81,1.2,1.8,2.2,2.7,3.3,4.7
0,AUC,0.319853,0.34421,0.674173,0.808824,0.824908,0.829963,0.83318,0.837316,0.840533,0.840533,0.84329,0.84329,0.84329,0.84329,0.84329,0.850643,0.850643
1,Epochs,37.000000,26.00000,20.000000,20.000000,20.000000,20.000000,20.00000,20.000000,32.000000,20.000000,37.00000,21.00000,20.00000,32.00000,47.00000,22.000000,32.000000


Es interesante observar como para los learning rate mas chicos, el entrenamiento se corta debido al early stoping en los primeros 15 epochs, ya que dentro de esos primeros 15 las mejoras son pocas debido a su avance "suave", en comparación con los learning rates mas altos que obtienen una mejor optimización en mayor cantidad de epochs

In [24]:
# Cargo el mejor modelo entrenado
learning.load_weights(join(checkpoints_path, 'learning'))
verify_model(learning, x_train_norm, y_train, x_valid_norm, y_valid)

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.825399,0.725762,0.740933,0.590909,0.839744
1,Validacion,0.850643,0.765625,0.794118,0.642857,0.875000


#### Scheduling learning rate

In [25]:
learning_schedule = ExponentialDecay(initial_learning_rate=2.7, decay_steps=100, decay_rate=0.96)
exponential_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'exponential'),
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)


In [26]:
learning = Sequential()
learning.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
learning.compile(optimizer=SGD(learning_rate=learning_schedule), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

In [27]:
history = learning.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[exponential_checkpoint_callback, stop_early], 
                      verbose=1)
# Cargo el mejor modelo entrenado
learning.load_weights(join(checkpoints_path, 'exponential'))
verify_model(learning, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/100
18/18 [==============================] - 1s 28ms/step - loss: 0.5559 - auc: 0.7777 - val_loss: 0.6140 - val_auc: 0.7530
Epoch 2/100
18/18 [==============================] - 0s 16ms/step - loss: 0.5113 - auc: 0.8140 - val_loss: 0.5270 - val_auc: 0.7969
Epoch 3/100
18/18 [==============================] - 0s 22ms/step - loss: 0.5199 - auc: 0.8058 - val_loss: 0.5177 - val_auc: 0.8300
Epoch 4/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5265 - auc: 0.8058 - val_loss: 0.5168 - val_auc: 0.7900
Epoch 5/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5209 - auc: 0.8075 - val_loss: 0.5566 - val_auc: 0.7983
Epoch 6/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5154 - auc: 0.8175 - val_loss: 0.7936 - val_auc: 0.5269
Epoch 7/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5382 - auc: 0.7934 - val_loss: 0.4953 - val_auc: 0.8095
Epoch 8/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5233 

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.802693,0.839335,0.569948,0.654762,0.784974
1,Validacion,0.840533,0.875000,0.588235,0.714286,0.800000


### Momentum

In [28]:
momentum_arr = [0.5, 0.6, 0.85, 0.9, 0.95, 0.99, 0.999]

arr_of_metrics = []
auc_res = {'momentum':['AUC', 'Epochs']}
for i in momentum_arr:
    momentum_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'momentum_{}'.format(i)),
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)
    
    mom_model = Sequential()
    mom_model.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
    mom_model.compile(optimizer=SGD(learning_rate=learning_schedule, momentum=i), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

    history = mom_model.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[momentum_checkpoint_callback], 
                          verbose=1)
    # Cargo el mejor modelo entrenado
    mom_model.load_weights(join(checkpoints_path, 'momentum_{}'.format(i)))
    metrics = verify_model(mom_model, x_train_norm, y_train, x_valid_norm, y_valid)
    arr_of_metrics.append(metrics)
    auc_res[i] = [metrics['AUC ROC'][1], history.epoch[-1]]
    clear_output(wait=True)
pd.DataFrame(data=auc_res)

,momentum,0.5,0.6,0.85,0.9,0.95,0.99,0.999
0,AUC,0.84421,0.846967,0.839614,0.841912,0.83341,0.826287,0.811581
1,Epochs,99.00000,99.000000,99.000000,99.000000,99.00000,99.000000,99.000000


In [29]:
momentum_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'momentum'),
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)

mom_model = Sequential()
mom_model.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
mom_model.compile(optimizer=SGD(learning_rate=learning_schedule, momentum=0.6), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])
history = mom_model.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[momentum_checkpoint_callback, stop_early], 
                          verbose=1)
# Cargo el mejor modelo entrenado
mom_model.load_weights(join(checkpoints_path, 'momentum'))
verify_model(mom_model, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/100
18/18 [==============================] - 0s 27ms/step - loss: 0.6632 - auc: 0.7718 - val_loss: 0.6448 - val_auc: 0.7376
Epoch 2/100
18/18 [==============================] - 0s 19ms/step - loss: 0.6021 - auc: 0.7861 - val_loss: 0.6060 - val_auc: 0.7456
Epoch 3/100
18/18 [==============================] - 0s 14ms/step - loss: 0.5783 - auc: 0.7830 - val_loss: 0.6408 - val_auc: 0.7489
Epoch 4/100
18/18 [==============================] - 0s 17ms/step - loss: 0.5811 - auc: 0.7839 - val_loss: 0.5723 - val_auc: 0.8235
Epoch 5/100
18/18 [==============================] - 0s 2ms/step - loss: 0.6263 - auc: 0.7773 - val_loss: 0.7001 - val_auc: 0.7063
Epoch 6/100
18/18 [==============================] - 0s 2ms/step - loss: 0.6500 - auc: 0.7579 - val_loss: 0.5991 - val_auc: 0.7893
Epoch 7/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5997 - auc: 0.7809 - val_loss: 0.6206 - val_auc: 0.7822
Epoch 8/100
18/18 [==============================] - 0s 2ms/step - loss: 0.6633

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.801501,0.878116,0.523316,0.696552,0.775061
1,Validacion,0.839154,0.906250,0.588235,0.769231,0.805556


### Optimizador Adam

In [30]:
%%time
adam_lr = [1e-5, 1e-4, 1e-3, 0.01, 0.05, 0.1, 0.22, 0.33, 0.47, 0.56, 0.81, 1.2, 1.8, 2.2, 2.7, 3.3, 4.7]

arr_of_metrics = []
auc_res = {'L_r':['AUC', 'Epochs']}
for i in adam_lr:
    adam_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'adam_{}'.format(i)),
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)
    adam_mod = Sequential()
    adam_mod.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
    adam_mod.compile(optimizer=Adam(learning_rate=i), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

    history = adam_mod.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[adam_checkpoint_callback], 
                          verbose=1, use_multiprocessing=True)
    # Cargo el mejor modelo entrenado
    adam_mod.load_weights(join(checkpoints_path, 'adam_{}'.format(i)))
    metrics = verify_model(adam_mod, x_train_norm, y_train, x_valid_norm, y_valid)
    arr_of_metrics.append(metrics)
    auc_res[i] = [metrics['AUC ROC'][1], history.epoch[-1]]
    clear_output(wait=True)
pd.DataFrame(data=auc_res)

Wall time: 2min 4s


,L_r,1e-05,0.0001,0.001,0.01,0.05,0.1,0.22,0.33,0.47,0.56,0.81,1.2,1.8,2.2,2.7,3.3,4.7
0,AUC,0.279412,0.668199,0.818015,0.835938,0.840074,0.84329,0.846048,0.846967,0.848346,0.84421,0.836397,0.839154,0.855239,0.837316,0.848805,0.837316,0.836857
1,Epochs,99.000000,99.000000,99.000000,99.000000,99.000000,99.00000,99.000000,99.000000,99.000000,99.00000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000


In [31]:
%%time
adam_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'adam'),
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)

adam_model = Sequential()
adam_model.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
adam_model.compile(optimizer=Adam(learning_rate=4.7), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])
history = adam_model.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[adam_checkpoint_callback], 
                          verbose=1)
# Cargo el mejor modelo entrenado
adam_model.load_weights(join(checkpoints_path, 'adam'))
verify_model(adam_model, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/100
18/18 [==============================] - 0s 22ms/step - loss: 2.7355 - auc: 0.7011 - val_loss: 2.1783 - val_auc: 0.6643
Epoch 2/100
18/18 [==============================] - 0s 10ms/step - loss: 2.3070 - auc: 0.7135 - val_loss: 1.8062 - val_auc: 0.7511
Epoch 3/100
18/18 [==============================] - 0s 2ms/step - loss: 2.2435 - auc: 0.7079 - val_loss: 1.9042 - val_auc: 0.7484
Epoch 4/100
18/18 [==============================] - 0s 2ms/step - loss: 2.1299 - auc: 0.7381 - val_loss: 2.4922 - val_auc: 0.6710
Epoch 5/100
18/18 [==============================] - 0s 2ms/step - loss: 1.6313 - auc: 0.6939 - val_loss: 1.2450 - val_auc: 0.7482
Epoch 6/100
18/18 [==============================] - 0s 14ms/step - loss: 1.3831 - auc: 0.7281 - val_loss: 1.8159 - val_auc: 0.7601
Epoch 7/100
18/18 [==============================] - 0s 2ms/step - loss: 1.8237 - auc: 0.6833 - val_loss: 1.6740 - val_auc: 0.7578
Epoch 8/100
18/18 [==============================] - 0s 2ms/step - loss: 1.6052 

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.794318,0.739612,0.683938,0.584071,0.814024
1,Validacion,0.842371,0.734375,0.823529,0.622222,0.886792


Evaluación contra test

In [32]:
print('Utilizando SGD')
verify_model(mom_model, x_train_norm, y_train, x_test_norm, y_test, valid_label='Test')

Utilizando SGD


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.801501,0.878116,0.523316,0.696552,0.775061
1,Test,0.842927,0.906667,0.560976,0.766667,0.790698


In [33]:
print('Utilizando Adam')
verify_model(adam_model, x_train_norm, y_train, x_test_norm, y_test, valid_label='Test')

Utilizando Adam


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.794318,0.739612,0.683938,0.584071,0.814024
1,Test,0.836423,0.760000,0.780488,0.640000,0.863636


Finalmente se obtiene que el modelo que utiliza como optimizador SGD y los hyperparámetros optimizados fue el mejor de los entrenados hasta ahora

Cabe destacar una particularidad, que los resustados en Test son mejores que en el set de validacion, e incluso mejores que el set de train, con lo cual concluir que la metrica de nuestro modelo es AUC=0.86 no sería correcto, ya que es el caso en el que tenemos un estimador con alta varianza, por lo tanto una estimación utilizando k-folding sería mas adecuada (si hay tiempo la hago...)